In [34]:
# Call libraries
from Bio import PDB
import tempfile
import os

# Create function to extract coordinates
def get_coords(pdb_id, chain_id, residue_id):
    pdb_id = pdb_id.lower()
    pdbl = PDB.PDBList()

    # Create temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        pdb_file_path = pdbl.retrieve_pdb_file(pdb_id, pdir=temp_dir, file_format="pdb")

        # Parse the file
        parser = PDB.PDBParser(QUIET=True)
        structure = parser.get_structure(pdb_id, pdb_file_path)
    
        # Loop through file to extract residue coordinates
        for model in structure:
            if chain_id in model:
                chain = model[chain_id]
                if residue_id in chain:
                    residue = chain[residue_id]
                    for atom in residue:
                        if atom.get_name() == "CA":
                            return atom.coord

# Select pdb code, chain and residue
pdb_id = "2RH1"
chain_id = "A"
residue_id = 31

coordinates = get_coords(pdb_id, chain_id, residue_id)
print(coordinates)

[-55.757   1.618  20.079]
